- Log Action time count has a correlation of 0.69 with score
- and Log word Count are correlated with score with r = 0.64

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dask.distributed import Client
import dask.dataframe as dd

In [2]:
# client = Client(n_workers = 4, threads_per_worker = 1, memory_limit = "1GB")
# client

In [3]:
notes = set() 
input_folder = "kaggle/input/linking-writing-processes-to-writing-quality/"

In [4]:
train_logs = dd.read_csv(input_folder + "train_logs.csv",delimiter = ",",header = 0)
train_scores = pd.read_csv(input_folder +"train_scores.csv", delimiter = ",", header = 0)

In [23]:
train_logs.head(10)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1
5,001519c8,6,107296,107400,104,Input,q,q,q,3,1
6,001519c8,7,107469,107596,127,Input,q,q,q,4,1
7,001519c8,8,107659,107766,107,Input,q,q,q,5,1
8,001519c8,9,107743,107852,109,Input,q,q,q,6,1
9,001519c8,10,107840,107978,138,Input,Space,Space,,7,1


In [6]:
train_scores.head()

,id,score
0,001519c8,3.5
1,0022f953,3.5
2,0042269b,6.0
3,0059420b,2.0
4,0075873a,4.0


## **Get maximum word count per essay**
Below, we computet the log of the maximum word count and found out that its correlation with score increase from 0.63(non log) to 0.69 (log applied)

In [19]:
word_count = train_logs.groupby(['id'])['word_count'].max().compute()
log_word_count = np.log(word_count)
score = pd.Series(data = train_scores.score.values, index = train_scores.id, name = "score")

In [124]:
z = pd.merge(log_word_count, score, left_index = True, right_index = True)
z.columns = ['log_word_count', 'score']
z.head()

,log_word_count,score
id,,
001519c8,5.545177,3.5
0022f953,5.777652,3.5
0042269b,6.001415,6.0
0059420b,5.327876,2.0
0075873a,5.529429,4.0


In [125]:
z.corr()

,log_word_count,score
log_word_count,1.000000,0.691226
score,0.691226,1.000000


## **Checkout the number of action time per id** 
If we log the number of actions, the correlation improved from 0.59 to 0.64

In [126]:
n_action_time_stats = train_logs.groupby(['id'])['action_time'].aggregate(['count','max']).compute()

In [127]:
n_action_time_stats['log_action_time_count'] = np.log(n_action_time_stats['count'])
n_action_time_stats.drop(labels = "count", inplace = True, axis = 1)


In [129]:
z = pd.merge(z,n_action_time_stats, left_index = True,right_index = True) 

In [130]:
z.corr()

,log_word_count,score,max,log_action_time_count
log_word_count,1.000000,0.691226,0.051713,0.836508
score,0.691226,1.000000,0.065637,0.646826
max,0.051713,0.065637,1.000000,0.061137
log_action_time_count,0.836508,0.646826,0.061137,1.000000
